In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

## 1. 데이터 불러오기 (2021 기준)
    - 1-1. 휴장일
    - 1-2. Newsfile 
    - 1-3. 반도체(005290) : 상승률 1.5 이상--1, 상승률 1.5 이하--0 라벨링 데이터

### 1-1. 2021 휴장일 날짜

In [2]:
closedate_2021 = pd.read_excel('../data/closedate_2021.xls')
closedate_2021.head(3)

,일자 및 요일,요일구분,비고
0,2021-01-01,금요일,신정
1,2021-02-11,목요일,설날
2,2021-02-12,금요일,설날


In [3]:
closedate_2021 = closedate_2021[['일자 및 요일', '요일구분']]
closedate_2021.head(3)

,일자 및 요일,요일구분
0,2021-01-01,금요일
1,2021-02-11,목요일
2,2021-02-12,금요일


### 1-2. newsfile

- train_data

In [6]:
news_train = pd.read_csv('../data/train_data_hanja_translate.csv')
news_train.head(3)

,Unnamed: 0,date,title,nouns
0,37650,20190109,작년 분기 어닝 쇼크 삼성전자글로벌 반도체 왕좌 다시 내주나,작년 분기 어닝 쇼크 삼성 전자 글로벌 반도체 왕좌 다시 내주
1,9503,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
2,29948,20190806,사진반도체 공장 둘러보는 이재용 부회장,사진 반도체 공장 이재용 부회장


In [8]:
news_train = news_train[['date', 'title', 'nouns']]
news_train.head(3)

,date,title,nouns
0,20190109,작년 분기 어닝 쇼크 삼성전자글로벌 반도체 왕좌 다시 내주나,작년 분기 어닝 쇼크 삼성 전자 글로벌 반도체 왕좌 다시 내주
1,20210714,삼성전자카이스트 반도체 계약학과 신설인재 양성 가속,삼성 전자 카이스트 반도체 계약 학과 신설 인재 양성 가속
2,20190806,사진반도체 공장 둘러보는 이재용 부회장,사진 반도체 공장 이재용 부회장


- test_data

In [7]:
news_test = pd.read_csv('../data/test_data_hanja_translate.csv')
news_test.head(3)

,Unnamed: 0,date,title,nouns
0,28288,20191008,리탈 제회 반도체대전SEDEX 참가,리탈 회 반도체 대전 참가
1,3825,20211130,정부 반도체용 수입 산업가스 수급 안정화고압가스 반송기한 최대 년으로 연,정부 반도체 용 수입 산업 가스 수급 안정화 고압 가스 반송 기한 최대 년 연
2,11223,20210524,김대호의 경제읽기 굳건해진 반도체배터리 협력경제동맹 새 지평,김대호 경제 해진 반도체 배터리 협력 경제 동맹 새 지평


In [10]:
news_test = news_test[['date', 'title', 'nouns']]
news_test.head(3)

,date,title,nouns
0,20191008,리탈 제회 반도체대전SEDEX 참가,리탈 회 반도체 대전 참가
1,20211130,정부 반도체용 수입 산업가스 수급 안정화고압가스 반송기한 최대 년으로 연,정부 반도체 용 수입 산업 가스 수급 안정화 고압 가스 반송 기한 최대 년 연
2,20210524,김대호의 경제읽기 굳건해진 반도체배터리 협력경제동맹 새 지평,김대호 경제 해진 반도체 배터리 협력 경제 동맹 새 지평
